### Capstone Project Introduction  - Analysing the optimal neighbourhood to open a new restaurant in the Manhattan Borough of New York using Foursquare and New York Data 

Using foursquare location data and New York City postcode/neighbourhood data I will investigating the most popular resturant type in the Manhattan borough and using this information to Identify the optimal neighbourhood to open a new resturant. To assess this problem I will use clustering and segmentation techniques to help assign the venues into classes, this will help me to assess which resturants are most popular in Manhattan as a whole and then in each neighbourhood of Manhattan. I will be looking at the frequency of restaurants as an indicator of which neighbourhood would be a good candidate for a new resturant, and also the popularity of each type of restaurant. I will then make a recommendation as to which type of restaurant may be favourable to open and the area in Manhattan that would be optimal to open this restaurant. 